## DIA 023: Mejora de Seguridad con Autenticación JWT y Rate Limiting


### **Índice**
1. [Objetivos del Día 23](#1-objetivos-del-día-23)
2. [Implementar Autenticación JWT](#2-implementar-autenticación-jwt)
   - [2.1. Instalar Dependencias Necesarias](#21-instalar-dependencias-necesarias)
   - [2.2. Configurar JWT en `api.py`](#22-configurar-jwt-en-apipy)
   - [2.3. Crear Endpoints de Autenticación](#23-crear-endpoints-de-autenticación)
   - [2.4. Actualizar el Frontend para Manejar JWT](#24-actualizar-el-frontend-para-manejar-jwt)
3. [Implementar Rate Limiting](#3-implementar-rate-limiting)
   - [3.1. Instalar Flask-Limiter](#31-instalar-flask-limiter)
   - [3.2. Configurar Rate Limiting en `api.py`](#32-configurar-rate-limiting-en-apipy)
4. [Actualizar las Pruebas Automatizadas](#4-actualizar-las-pruebas-automatizadas)
   - [4.1. Actualizar `test_api.py` para JWT](#41-actualizar-test_apipy-para-jwt)
   - [4.2. Añadir Pruebas para Rate Limiting](#42-añadir-pruebas-para-rate-limiting)
5. [Conclusiones y Recomendaciones](#5-conclusiones-y-recomendaciones)
6. [Recursos Adicionales](#6-recursos-adicionales)

---

## **1. Objetivos del Día 23**

1. **Implementar Autenticación JWT:**
   - Proveer una forma segura de autenticar a los usuarios que acceden a la API.
   - Reemplazar el uso de API Keys estáticas por tokens JWT dinámicos.

2. **Implementar Rate Limiting:**
   - Restringir la cantidad de solicitudes que un usuario puede hacer en un período de tiempo determinado.
   - Proteger la API contra abusos y ataques de denegación de servicio (DoS).

3. **Actualizar las Pruebas Automatizadas:**
   - Asegurar que los nuevos mecanismos de autenticación y rate limiting funcionan correctamente.

---

## **2. Implementar Autenticación JWT**

La autenticación JWT (JSON Web Tokens) permite que los usuarios obtengan un token seguro que deben incluir en sus solicitudes a la API. Este token autentica al usuario y puede contener información adicional, como roles o permisos.

### **2.1. Instalar Dependencias Necesarias**

Añade las siguientes dependencias a tu archivo `requirements.txt`:

```plaintext
Flask-JWT-Extended==4.4.4
```

**Instalar las Dependencias:**

```bash
pip install -r requirements.txt
```

### **2.2. Configurar JWT en `api.py`**

Actualiza tu archivo `api.py` para configurar Flask-JWT-Extended.

```python
# api.py

from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from PIL import Image
import numpy as np
import io
import os
import redis
import hashlib
from functools import wraps
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity

app = Flask(__name__)

# Configuración de JWT
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'super-secret-key')  # Cambia esto a una clave segura
jwt = JWTManager(app)

# Configurar Redis
redis_url = os.getenv('REDIS_URL', 'redis://localhost:6379/0')
cache = redis.Redis.from_url(redis_url)

# Cargar el modelo optimizado
modelo_web = tf.keras.models.load_model('modelo_cnn_transfer_learning_mnist_final_optimizado.h5')

# Obtener la API Key desde variables de entorno (mantendremos la API Key para autenticación de desarrollo)
API_KEY = os.getenv('API_KEY')

def require_api_key(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        key = request.headers.get('x-api-key')
        if key and key == API_KEY:
            return func(*args, **kwargs)
        else:
            return jsonify({'error': 'Unauthorized access'}), 401
    return wrapper

def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def generate_cache_key(file_bytes):
    return hashlib.sha256(file_bytes).hexdigest()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/login', methods=['POST'])
def login():
    """
    Endpoint para que los usuarios inicien sesión y obtengan un token JWT.
    En un escenario real, deberías validar las credenciales contra una base de datos.
    Aquí, para simplificar, aceptaremos cualquier usuario.
    """
    username = request.json.get('username', None)
    password = request.json.get('password', None)
    
    # Validar las credenciales (aquí simplemente verificamos que existan)
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    
    # Crear el token de acceso
    access_token = create_access_token(identity=username)
    return jsonify(access_token=access_token), 200

@app.route('/predict', methods=['POST'])
@jwt_required()
def predict_web():
    current_user = get_jwt_identity()
    
    if 'file' not in request.files:
        return jsonify({'error': 'No se encontró el archivo en la solicitud.'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No se seleccionó ningún archivo.'}), 400
    if not allowed_file(file.filename):
        return jsonify({'error': 'Tipo de archivo no permitido. Por favor, sube una imagen PNG, JPG, JPEG o GIF.'}), 400
    try:
        # Leer los bytes del archivo para generar la clave de cache
        file_bytes = file.read()
        cache_key = generate_cache_key(file_bytes)
        
        # Verificar si la predicción ya está en cache
        cached_result = cache.get(cache_key)
        if cached_result:
            prediccion, probabilidad = cached_result.decode('utf-8').split(',')
            return jsonify({'prediccion': int(prediccion), 'probabilidad': float(probabilidad), 'cached': True})
        
        # Procesar la imagen
        img = Image.open(io.BytesIO(file_bytes)).convert('L')  # Convertir a escala de grises
        img = img.resize((28, 28))  # Redimensionar a 28x28
        img_array = np.array(img).astype('float32') / 255.0  # Normalizar
        img_array = img_array.reshape((1, 28, 28, 1))  # Añadir dimensión de batch

        # Preprocesar para el modelo (redimensionar a 224x224 y convertir a RGB)
        img_resized = tf.image.resize(img_array, [224, 224])
        img_rgb = tf.image.grayscale_to_rgb(img_resized)

        # Realizar la predicción
        pred = modelo_web.predict(img_rgb)
        pred_class = np.argmax(pred, axis=1)[0]
        pred_prob = float(np.max(pred))

        # Almacenar el resultado en cache por 1 hora
        cache.setex(cache_key, 3600, f"{pred_class},{pred_prob}")

        return jsonify({'prediccion': int(pred_class), 'probabilidad': pred_prob, 'cached': False})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)
```

**Explicación:**
- **Configuración de JWT:**
  - Se configura `JWT_SECRET_KEY` para firmar los tokens JWT. Es crucial cambiar esta clave a una más segura en producción y mantenerla en una variable de entorno.
  - Se inicializa `JWTManager` con la aplicación Flask.
  
- **Endpoint `/login`:**
  - Permite a los usuarios autenticarse y obtener un token JWT.
  - En un entorno real, deberías verificar las credenciales contra una base de datos de usuarios. Aquí, simplemente aceptamos cualquier usuario que proporcione un nombre de usuario y contraseña.
  
- **Protección del Endpoint `/predict`:**
  - Se utiliza el decorador `@jwt_required()` para proteger el endpoint.
  - Se obtiene la identidad del usuario con `get_jwt_identity()`.

### **2.3. Crear Endpoints de Autenticación**

Ya se ha creado el endpoint `/login` para obtener el token JWT. Ahora, actualizaremos el frontend para que los usuarios puedan iniciar sesión y usar el token para acceder a la API.

### **2.4. Actualizar el Frontend para Manejar JWT**

Actualiza tu archivo `index.html` para incluir un formulario de inicio de sesión y manejar tokens JWT.

```html
<!-- templates/index.html -->

<!DOCTYPE html>
<html>
<head>
    <title>Clasificador de Dígitos MNIST</title>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
</head>
<body>

    <h1>Clasificador de Dígitos MNIST</h1>
    
    <!-- Formulario de Login -->
    <h2>Iniciar Sesión</h2>
    <form id="loginForm">
        <label for="username">Usuario:</label>
        <input type="text" id="username" name="username" required>
        <br>
        <label for="password">Contraseña:</label>
        <input type="password" id="password" name="password" required>
        <br>
        <button type="submit">Iniciar Sesión</button>
    </form>
    
    <div id="loginResult"></div>
    
    <!-- Formulario de Predicción -->
    <h2>Enviar Imagen para Predicción</h2>
    <input type="file" id="fileInput" accept="image/*">
    <button onclick="uploadImage()">Enviar para Predicción</button>

    <h2>Resultado:</h2>
    <p id="result"></p>

    <script>
        let accessToken = '';

        // Manejar el formulario de login
        $('#loginForm').submit(function(event) {
            event.preventDefault();
            const username = $('#username').val();
            const password = $('#password').val();

            $.ajax({
                url: '/login',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, password: password }),
                success: function(response) {
                    accessToken = response.access_token;
                    $('#loginResult').text('Inicio de sesión exitoso. Token obtenido.');
                },
                error: function(xhr, status, error) {
                    $('#loginResult').text('Error en el inicio de sesión: ' + xhr.responseJSON.msg);
                }
            });
        });

        function uploadImage() {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            var fileInput = document.getElementById('fileInput');
            var file = fileInput.files[0];
            if (!file) {
                alert("Por favor, selecciona una imagen.");
                return;
            }

            var formData = new FormData();
            formData.append('file', file);

            $.ajax({
                url: '/predict',
                type: 'POST',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                data: formData,
                contentType: false,
                processData: false,
                success: function(response) {
                    if(response.error){
                        $('#result').text('Error: ' + response.error);
                    } else {
                        let cacheStatus = response.cached ? ' (Desde Cache)' : '';
                        $('#result').text('Predicción: ' + response.prediccion + ' | Probabilidad: ' + (response.probabilidad * 100).toFixed(2) + '%' + cacheStatus);
                    }
                },
                error: function(xhr, status, error) {
                    $('#result').text('Error: ' + xhr.responseJSON.error);
                }
            });
        }
    </script>

</body>
</html>
```

**Explicación:**
- **Formulario de Login:**
  - Permite a los usuarios ingresar su nombre de usuario y contraseña.
  - Al enviar el formulario, se realiza una solicitud POST al endpoint `/login` para obtener un token JWT.
  
- **Manejo del Token JWT:**
  - Si el login es exitoso, se almacena el token en la variable `accessToken`.
  - Este token se utiliza en las solicitudes al endpoint `/predict` mediante el encabezado `Authorization: Bearer <token>`.
  
- **Formulario de Predicción:**
  - Solo permite enviar una imagen si el usuario ha iniciado sesión y posee un token válido.
  - Muestra si la respuesta proviene del caché con el indicador `(Desde Cache)`.

### **2.5. Actualizar Variables de Entorno en Heroku**

Para mayor seguridad, es recomendable mover la clave secreta de JWT a una variable de entorno en Heroku.

1. **Establecer `JWT_SECRET_KEY` en Heroku:**

   ```bash
   heroku config:set JWT_SECRET_KEY=tu_clave_secreta_segura --app mnist-flask-api
   ```

   **Nota:** Reemplaza `tu_clave_secreta_segura` con una clave fuerte y única.

2. **Reiniciar la Aplicación:**

   ```bash
   heroku restart --app mnist-flask-api
   ```

### **2.6. Probar la Autenticación JWT**

1. **Iniciar Sesión en el Frontend:**
   - Abre tu aplicación en el navegador.
   - Ingresa un nombre de usuario y contraseña en el formulario de login.
   - Al iniciar sesión correctamente, recibirás un mensaje indicando que el token ha sido obtenido.

2. **Enviar una Imagen para Predicción:**
   - Selecciona una imagen y haz clic en "Enviar para Predicción".
   - Deberías ver la predicción junto con la probabilidad y un indicador si el resultado proviene del caché.

3. **Probar Solicitudes sin Token:**
   - Intenta enviar una solicitud de predicción sin haber iniciado sesión.
   - Deberías recibir un error de acceso no autorizado.

---

## **3. Implementar Rate Limiting**

El rate limiting restringe la cantidad de solicitudes que un usuario puede hacer en un período de tiempo determinado, protegiendo la API contra abusos y ataques de denegación de servicio (DoS).

### **3.1. Instalar Flask-Limiter**

Añade `Flask-Limiter` a tu archivo `requirements.txt`:

```plaintext
Flask-Limiter==2.8.1
```

**Instalar las Dependencias:**

```bash
pip install -r requirements.txt
```

### **3.2. Configurar Rate Limiting en `api.py`**

Actualiza tu archivo `api.py` para integrar Flask-Limiter.

```python
# api.py

from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from PIL import Image
import numpy as np
import io
import os
import redis
import hashlib
from functools import wraps
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

app = Flask(__name__)

# Configuración de JWT
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'super-secret-key')  # Cambia esto a una clave segura
jwt = JWTManager(app)

# Configurar Rate Limiting
limiter = Limiter(
    app,
    key_func=get_remote_address,
    default_limits=["200 per day", "50 per hour"]
)

# Configurar Redis
redis_url = os.getenv('REDIS_URL', 'redis://localhost:6379/0')
cache = redis.Redis.from_url(redis_url)

# Cargar el modelo optimizado
modelo_web = tf.keras.models.load_model('modelo_cnn_transfer_learning_mnist_final_optimizado.h5')

# Obtener la API Key desde variables de entorno (mantendremos la API Key para autenticación de desarrollo)
API_KEY = os.getenv('API_KEY')

def require_api_key(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        key = request.headers.get('x-api-key')
        if key and key == API_KEY:
            return func(*args, **kwargs)
        else:
            return jsonify({'error': 'Unauthorized access'}), 401
    return wrapper

def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def generate_cache_key(file_bytes):
    return hashlib.sha256(file_bytes).hexdigest()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/login', methods=['POST'])
def login():
    """
    Endpoint para que los usuarios inicien sesión y obtengan un token JWT.
    En un escenario real, deberías validar las credenciales contra una base de datos.
    Aquí, para simplificar, aceptaremos cualquier usuario.
    """
    username = request.json.get('username', None)
    password = request.json.get('password', None)
    
    # Validar las credenciales (aquí simplemente verificamos que existan)
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    
    # Crear el token de acceso
    access_token = create_access_token(identity=username)
    return jsonify(access_token=access_token), 200

@app.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("10 per minute")  # Limita a 10 solicitudes por minuto por usuario
def predict_web():
    current_user = get_jwt_identity()
    
    if 'file' not in request.files:
        return jsonify({'error': 'No se encontró el archivo en la solicitud.'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No se seleccionó ningún archivo.'}), 400
    if not allowed_file(file.filename):
        return jsonify({'error': 'Tipo de archivo no permitido. Por favor, sube una imagen PNG, JPG, JPEG o GIF.'}), 400
    try:
        # Leer los bytes del archivo para generar la clave de cache
        file_bytes = file.read()
        cache_key = generate_cache_key(file_bytes)
        
        # Verificar si la predicción ya está en cache
        cached_result = cache.get(cache_key)
        if cached_result:
            prediccion, probabilidad = cached_result.decode('utf-8').split(',')
            return jsonify({'prediccion': int(prediccion), 'probabilidad': float(probabilidad), 'cached': True})
        
        # Procesar la imagen
        img = Image.open(io.BytesIO(file_bytes)).convert('L')  # Convertir a escala de grises
        img = img.resize((28, 28))  # Redimensionar a 28x28
        img_array = np.array(img).astype('float32') / 255.0  # Normalizar
        img_array = img_array.reshape((1, 28, 28, 1))  # Añadir dimensión de batch

        # Preprocesar para el modelo (redimensionar a 224x224 y convertir a RGB)
        img_resized = tf.image.resize(img_array, [224, 224])
        img_rgb = tf.image.grayscale_to_rgb(img_resized)

        # Realizar la predicción
        pred = modelo_web.predict(img_rgb)
        pred_class = np.argmax(pred, axis=1)[0]
        pred_prob = float(np.max(pred))

        # Almacenar el resultado en cache por 1 hora
        cache.setex(cache_key, 3600, f"{pred_class},{pred_prob}")

        return jsonify({'prediccion': int(pred_class), 'probabilidad': pred_prob, 'cached': False})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)
```

**Explicación:**
- **Configuración de Rate Limiting:**
  - Se inicializa `Limiter` con Flask-Limiter, utilizando la dirección IP remota como clave para limitar las solicitudes.
  - Se establecen límites predeterminados de 200 solicitudes por día y 50 por hora.
  
- **Rate Limiting Específico por Endpoint:**
  - En el endpoint `/predict`, se aplica un límite adicional de 10 solicitudes por minuto por usuario.
  
### **3.3. Personalizar los Mensajes de Rate Limiting (Opcional)**

Puedes personalizar los mensajes que se devuelven cuando se supera el límite de solicitudes.

```python
# api.py

from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from PIL import Image
import numpy as np
import io
import os
import redis
import hashlib
from functools import wraps
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

app = Flask(__name__)

# Configuración de JWT
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'super-secret-key')  # Cambia esto a una clave segura
jwt = JWTManager(app)

# Configurar Rate Limiting
limiter = Limiter(
    app,
    key_func=get_remote_address,
    default_limits=["200 per day", "50 per hour"],
    default_limits_exempt=["/login"]  # Exento el endpoint de login
)

# Personalizar el mensaje de Rate Limiting
@app.errorhandler(429)
def ratelimit_handler(e):
    return jsonify(error="Has superado el límite de solicitudes. Por favor, intenta nuevamente más tarde."), 429

# Resto del código permanece igual
```

**Explicación:**
- **Exención de Endpoints:** Se exime el endpoint `/login` de los límites de rate limiting para permitir que los usuarios obtengan tokens sin restricciones.
- **Personalización del Mensaje de Error:** Se define un manejador de errores para el código 429 (Too Many Requests) que devuelve un mensaje personalizado.

---

## **4. Actualizar las Pruebas Automatizadas**

Con la implementación de JWT y rate limiting, es necesario actualizar las pruebas automatizadas para asegurar que estos mecanismos funcionan correctamente.

### **4.1. Actualizar `test_api.py` para JWT**

Actualiza el archivo `tests/test_api.py` para incluir pruebas de autenticación JWT.

```python
# tests/test_api.py

import pytest
from api import app
import os
from io import BytesIO
from PIL import Image
import numpy as np
import json

@pytest.fixture
def client():
    app.config['TESTING'] = True
    with app.test_client() as client:
        yield client

def generate_image(digit=5):
    # Genera una imagen en blanco con un dígito negro en el centro
    img = Image.new('L', (28, 28), color=255)
    # Aquí podrías dibujar el dígito, pero para simplicidad, dejaremos la imagen en blanco
    # En un caso real, usarías una imagen real de MNIST
    return img

def login(client, username, password):
    response = client.post('/login', json={
        'username': username,
        'password': password
    })
    data = response.get_json()
    return data.get('access_token')

def test_predict_success(client):
    # Obtener el token de acceso
    token = login(client, 'testuser', 'testpassword')
    assert token is not None, "No se obtuvo el token JWT."
    
    img = generate_image()
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    data = {
        'file': (img_byte_arr, 'test.png')
    }
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 200
    json_data = response.get_json()
    assert 'prediccion' in json_data
    assert 'probabilidad' in json_data
    assert 'cached' in json_data
    assert isinstance(json_data['prediccion'], int)
    assert isinstance(json_data['probabilidad'], float)

def test_predict_no_file(client):
    # Obtener el token de acceso
    token = login(client, 'testuser', 'testpassword')
    assert token is not None, "No se obtuvo el token JWT."
    
    data = {}
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 400
    json_data = response.get_json()
    assert json_data['error'] == 'No se encontró el archivo en la solicitud.'

def test_predict_invalid_file_type(client):
    # Obtener el token de acceso
    token = login(client, 'testuser', 'testpassword')
    assert token is not None, "No se obtuvo el token JWT."
    
    # Crear un archivo de texto en lugar de una imagen
    file_data = BytesIO(b"Este no es una imagen válida.")
    data = {
        'file': (file_data, 'test.txt')
    }
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 400
    json_data = response.get_json()
    assert json_data['error'] == 'Tipo de archivo no permitido. Por favor, sube una imagen PNG, JPG, JPEG o GIF.'

def test_predict_unauthorized(client):
    # Intentar acceder al endpoint sin token
    img = generate_image()
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    data = {
        'file': (img_byte_arr, 'test.png')
    }
    
    response = client.post('/predict', data=data, content_type='multipart/form-data')
    assert response.status_code == 401
    json_data = response.get_json()
    assert json_data['error'] == 'Unauthorized access'

def test_rate_limiting(client):
    # Obtener el token de acceso
    token = login(client, 'testuser', 'testpassword')
    assert token is not None, "No se obtuvo el token JWT."
    
    img = generate_image()
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    data = {
        'file': (img_byte_arr, 'test.png')
    }
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    # Realizar 10 solicitudes válidas
    for _ in range(10):
        response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
        assert response.status_code in [200, 429]
        if response.status_code == 429:
            break
    
    # La siguiente solicitud debería exceder el límite
    response = client.post('/predict', data=data, headers=headers, content_type='multipart/form-data')
    assert response.status_code == 429
    json_data = response.get_json()
    assert json_data['error'] == 'Has superado el límite de solicitudes. Por favor, intenta nuevamente más tarde.'
```

**Explicación:**
- **Función `login`:**
  - Facilita la obtención de un token JWT para las pruebas.
  
- **Pruebas Actualizadas:**
  - **`test_predict_success`:** Ahora incluye la obtención y uso del token JWT.
  - **`test_predict_unauthorized`:** Verifica que acceder al endpoint sin token JWT retorna un error de acceso no autorizado.
  - **`test_rate_limiting`:** Realiza múltiples solicitudes para probar el rate limiting y asegura que se devuelve un error 429 al exceder el límite.

### **4.2. Añadir Pruebas para Rate Limiting**

La prueba `test_rate_limiting` en el archivo `test_api.py` ya cubre la verificación de rate limiting al realizar múltiples solicitudes y esperar que se alcance el límite.

---

## **5. Conclusiones y Recomendaciones**

### **5.1. Mejora de la Seguridad con JWT**
- **Beneficios:**
  - **Autenticación Segura:** Los tokens JWT son seguros y pueden contener información adicional sobre el usuario.
  - **Escalabilidad:** No requiere almacenamiento de sesiones en el servidor, facilitando la escalabilidad.
  
- **Recomendaciones:**
  - **Almacenar JWT en el Frontend de Forma Segura:** Evita exponer el token en lugares accesibles como el almacenamiento local. Considera usar cookies seguras y HTTP-only.
  - **Implementar Refresh Tokens:** Para mejorar la seguridad y la experiencia del usuario, implementa tokens de actualización que permitan renovar tokens de acceso expirados sin reingresar credenciales.

### **5.2. Implementación de Rate Limiting**
- **Beneficios:**
  - **Protección Contra Abusos:** Evita que un solo usuario o atacante sobrecargue la API con demasiadas solicitudes.
  - **Mejor Experiencia para Usuarios Legítimos:** Asegura que los usuarios legítimos puedan acceder a la API sin interferencias.
  
- **Recomendaciones:**
  - **Configurar Límites Apropiados:** Ajusta los límites según el uso esperado y el comportamiento de los usuarios.
  - **Monitorizar y Ajustar:** Observa el tráfico y ajusta los límites conforme crece la base de usuarios y cambia el uso de la API.
  
### **5.3. Actualización de Pruebas Automatizadas**
- **Beneficios:**
  - **Garantía de Calidad:** Asegura que las nuevas implementaciones no introduzcan errores.
  - **Facilidad de Mantenimiento:** Las pruebas documentan el comportamiento esperado y facilitan futuras modificaciones.
  
- **Recomendaciones:**
  - **Continuar Expandiendo las Pruebas:** Añade pruebas para nuevos endpoints o funcionalidades a medida que se desarrollan.
  - **Integrar Pruebas de Integración y de Carga:** Para evaluar cómo interactúan diferentes componentes de la API y cómo se comporta bajo carga.

---

## **6. Recursos Adicionales**

- **Flask-JWT-Extended Documentation:** [Flask-JWT-Extended Docs](https://flask-jwt-extended.readthedocs.io/en/stable/)
- **Flask-Limiter Documentation:** [Flask-Limiter Docs](https://flask-limiter.readthedocs.io/en/stable/)
- **PyTest Documentation:** [PyTest Docs](https://docs.pytest.org/en/7.1.x/)
- **GitHub Actions Documentation:** [GitHub Actions Docs](https://docs.github.com/en/actions)
- **Heroku Rate Limiting:** [Heroku Rate Limiting](https://devcenter.heroku.com/articles/rate-limiting)
- **Redis Documentation:** [Redis Docs](https://redis.io/documentation)
- **Flask Documentation:** [Flask Docs](https://flask.palletsprojects.com/en/2.0.x/)
- **TensorFlow Lite Model Optimization:** [TensorFlow Lite Optimization](https://www.tensorflow.org/lite/performance/model_optimization)
- **Celery Documentation:** [Celery Docs](https://docs.celeryproject.org/en/stable/)
- **Prometheus Documentation:** [Prometheus Docs](https://prometheus.io/docs/introduction/overview/)
- **Grafana Documentation:** [Grafana Docs](https://grafana.com/docs/grafana/latest/)
- **Loguru Documentation:** [Loguru Docs](https://loguru.readthedocs.io/en/stable/)

---

## **Conclusión**

En el **Día 23**, has fortalecido la **seguridad** de tu API mediante la **implementación de autenticación JWT** y la **introducción de rate limiting**. Estas mejoras aseguran que solo usuarios autorizados puedan acceder a la API y protegen contra abusos y ataques de denegación de servicio (DoS). Además, has actualizado las **pruebas automatizadas** para cubrir estos nuevos mecanismos, garantizando que funcionen correctamente.

**Pasos Clave Realizados:**
1. **Implementación de Autenticación JWT:**
   - Configuraste Flask-JWT-Extended para gestionar tokens de acceso.
   - Creaste un endpoint `/login` para que los usuarios obtengan tokens JWT.
   - Protegiste el endpoint `/predict` utilizando el decorador `@jwt_required()`.
   - Actualizaste el frontend para manejar el flujo de autenticación y uso de tokens.

2. **Implementación de Rate Limiting:**
   - Integraste Flask-Limiter para restringir la cantidad de solicitudes por usuario.
   - Configuraste límites generales y específicos para proteger endpoints críticos.
   - Personalizaste los mensajes de error para una mejor experiencia del usuario.

3. **Actualización de Pruebas Automatizadas:**
   - Añadiste pruebas para verificar la autenticación JWT.
   - Implementaste pruebas para asegurar que el rate limiting funciona como se espera.
   
**Recomendaciones para Continuar:**
- **Implementar Refresh Tokens:** Para mejorar la seguridad y la experiencia del usuario al manejar tokens expirados.
- **Optimizar el Frontend:** Implementa manejo de errores más robusto y almacenamiento seguro de tokens.
- **Monitorizar el Uso de la API:** Utiliza herramientas de monitoreo para observar patrones de tráfico y ajustar los límites de rate limiting según sea necesario.
- **Explorar Autenticación Avanzada:** Considera métodos como OAuth2 para integraciones más complejas y escalables.

¡Has realizado un excelente progreso en tu proyecto de **Transfer Learning**! Continúa aplicando estas prácticas de seguridad y optimización para mantener tu API robusta, segura y eficiente.

Si tienes alguna otra pregunta o necesitas más asistencia en los próximos pasos, **no dudes en contactarme**. ¡Estoy aquí para ayudarte a alcanzar el éxito en tu proyecto de **Transfer Learning**!